In [ ]:
import port_trad as ptd 
import pandas as pd
import port_trad.processing
import port_trad.scenario_generation
import math
from itertools import product
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
rs = 121

In [ ]:
from docplex.mp.model import Model

import scipy
import numpy as np

def distance_between(data1,series1):
    
    compares = []
    for i in range(data1.shape[0]):
        compares.append(scipy.spatial.distance.euclidean(data1.iloc[i,:],series1))
    locs = np.where(np.array(compares) == min(np.array(compares)))[0][0]
    return locs,data1.iloc[locs,:]

def adjust(x):
    xn = x.copy()
    n = len(xn)
    while True:
        if sum(xn<0)== 0:
            break

        negativex = []   
        positivex = []
        for i in range(n):
            if xn[i] > 0:
                positivex.append(xn[i])
            elif xn[i] < 0:
                negativex.append(xn[i])
        ad = sum(negativex)/len(positivex)

        for i in range(n):
            if xn[i] > 0:
                xn[i] = xn[i] + ad
                if (xn[i] <0) and (xn[i]> -0.000001):
                    xn[i] = 0
            elif xn[i] <0:
                xn[i] = 0
    return xn


In [ ]:
Asset = pd.read_csv("NASDAQ100_return.csv")
asset = Asset.drop("date", axis = 1)
assets,TMCADS = ptd.processing.asset_technical_indicators(asset)
assets.iloc[0:127]

In [ ]:
qw,_=distance_between(Asset.iloc[:,1:],assets.iloc[127,:])
Asset.iloc[qw,0]

In [ ]:
Time_window = 0
b = 0 + Time_window
e = 127 + Time_window
data = assets.iloc[b:e,:]
numbers = 1000
exo = TMCADS.iloc[b:e]
exo_predict = TMCADS.iloc[e]
N = 20 # prefer to make numbers/N = int
Cs = 50

# s1: Mean_reverting weighted 
scenario1 = ptd.scenario_generation.Mean_reverting_secnario(data,numbers, weit = "index", rstate = rs)

# s2: Mean_reverting non-weighted
scenario2 = ptd.scenario_generation.Mean_reverting_secnario(data,numbers, weit = None, rstate = rs)

# s3: GARCH(1,1) weighted
scenario3 = ptd.scenario_generation.CC_MGARCH_11(data,numbers,weit = 'index', rstate = rs)

# s4: GARCH(1,1) non-weighted
scenario4 = ptd.scenario_generation.CC_MGARCH_11(data,numbers,weit = None, rstate = rs)

# s5: CC_GJR_GARCH(1,1,1) weighted
scenario5 = ptd.scenario_generation.CC_GJR_GARCH(data,numbers, weit = 'index', rstate = rs)

# s6: CC_GJR_GARCH(1,1,1) non-weighted
scenario6 = ptd.scenario_generation.CC_GJR_GARCH(data,numbers, weit = None, rstate = rs)

# s7: CC_exo_GJR_GARCH(1,1,1) non-weighted
scenario7 = ptd.scenario_generation.exo_GJR_GARCH(data,numbers,exo,exo_predict,weit = 'index', rstate = rs)

# s8: CC_exo_GJR_GARCH(1,1,1) non-weighted
scenario8 = ptd.scenario_generation.exo_GJR_GARCH(data,numbers,exo,exo_predict,weit = None , rstate = rs)

#s9: multi-normal distributino
scenario9 = ptd.scenario_generation.Multi_norm_asset(data, numbers,rstate = rs)

#S10: multi-t distribution
scenario10 = ptd.scenario_generation.Multi_t_asset(data, numbers, rstate = rs, dfr = 4)

#S11: historial sampling
scenario11 = ptd.scenario_generation.historial_sampling(data,numbers,weit = None, rstate = rs)

#s12: last sampling
scenario12 = ptd.scenario_generation.sample_lastN(data,N )

scenario12 = pd.concat([scenario12]*math.ceil(numbers/N),ignore_index = True)


emad =0.25
q_var = 0.7

Scenarios_set = []
risk_measure = []
mean_value = []
for i in range(1,13):
    kmeans = KMeans(n_clusters= Cs, random_state=rs).fit(eval('scenario'+str(i)))
    clusters = kmeans.cluster_centers_
    clusters = pd.DataFrame(clusters,columns = eval('scenario'+str(i)).columns)
    # add risk_free asset in portfolio 
    clusters["risk_free"] = 0.03/365*7
    Scenarios_set.append(clusters.values.tolist())
    
    # add the risk measure
    risk_measure.append(ptd.processing.risk_mean_function(clusters.iloc[:,0],emad,q_var).values.tolist())
    
    # add the mean value 
    mean_value.append(clusters.mean().values.tolist())

return_target_set = np.array(Scenarios_set)[:,:,0]
Scenarios_set = np.array(Scenarios_set)[:,:,1:] # shape d,i,j (data set,scenario,asset)
risk_measure = np.array(risk_measure) # shape d,i,j (data set,risk_measure,1)
target_mean_value = np.array(mean_value)[:,0] # shape d,j (data set,asset)
assets_mean_value = np.array(mean_value)[:,1:] 

target_ratio = []
for i in range(risk_measure.shape[0]):
    if target_mean_value[i] <= 0:
        target_ratio.append(100000)
    else:
        target_ratio.append(((-risk_measure[:,0,0] +  risk_measure[:,2,0])/emad)[i]/target_mean_value[i])
        
        


In [ ]:
from docplex.mp.model import Model

# input the initial wealth 
# (if we make decision variable become amount rahter than weight, then we need to multiple them by initial welath to get correct target)
W0 = 100 # initial welath
gamma = 0.001 # transaction cost

# big M for chance constraint programming
M = 10000
beta = 0.7

T_sv = risk_measure[:,1,0] # "semivariance_loss"
T_EMAD = risk_measure[:,2,0] # "EMAD_loss"
T_VAR = risk_measure[:,3,0] # "VAR_loss"
T_CVAR = risk_measure[:,4,0] # "CVAR_loss"

# the number of data set 
sd = Scenarios_set.shape[0]
# the number of scenario in data set
si = Scenarios_set.shape[1]
# the number of asset
sj = Scenarios_set.shape[2]

# transaction cost
gamma_j = ([gamma]*(sj - 1))
gamma_j.append(0)

#(d,i,j)
index_dij =list(product(range(sd), range(si),range(sj)))
# (d,i)
index_di =list(product(range(sd), range(si)))
# （d,j)
index_dj =list(product(range(sd), range(sj)))
# （i,j)
index_ij =list(product(range(si), range(sj)))
#(d)
index_d = [i for i in range(sd)]
# (i)
index_i = [i for i in range(si)]
#(j)
index_j = [i for i in range(sj)]

# mu_dj 
mu_dj = {(d,j): assets_mean_value[d,j] for d,j in index_dj}
big_mu_j = {(j): pd.DataFrame(assets_mean_value).mean()[j] for j in index_j}

# return_rate(d,i,j)
r_dij = {(d,i,j): Scenarios_set[d,i,j] for d,i,j in index_dij}

# target value of portfolio of return in scenario d of asset i 
tv_di = {(d,i): W0*return_target_set[d,i] for d,i in index_di}

# the shape of risk measure is (d,n_th_risk_measure,1)
#the sequence of output of risk measure ["mean_loss","semivariance_loss","EMAD_loss","VAR_loss","CVAR_loss"]

t_sv = {(d): T_sv[d]*(W0**2) for d in index_d} # semivariance_loss"
t_EMAD = {(d): T_EMAD[d]*W0 for d in index_d}   # "EMAD_loss"
t_VAR = {(d): T_VAR[d]*W0 for d in index_d} # "VAR_loss"
t_CVAR = {(d): T_CVAR[d]*W0 for d in index_d} # "CVAR_loss"


In [ ]:
# Declare and initialize model
MON = Model(name = 'MO_N',log_output=True)

### decision variables

#the amount invested in asset j
k =  MON.continuous_var(lb=0.0, ub=float('inf'))
v_j =  MON.continuous_var_dict(index_j, lb=0.0, ub=float('inf'))
# auxiliary variable of MAD in data set d, scenario i
kem_di = MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
# auxiliary variable of CVaR in data set d, scenario i 
kcv_di =  MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
# auxiliary variable of semivariance in data set d, scenario i
ksv_di = MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
# the auxiliary variable of VaR 
kt_d = MON.continuous_var_dict(index_d, lb = -float('inf'),ub=float('inf'))
B1_d = MON.integer_var_dict(index_d,0,1)
B2_d = MON.integer_var_dict(index_d,0,1)

alpha1 = MON.continuous_var(lb=0.0,ub=float('inf'))
alpha2 = MON.continuous_var(lb=0.0,ub=float('inf'))
rf = -W0*gamma


for d in index_d:
    MON.add_constraint(MON.sum(big_mu_j[j]*v_j[j] for j in index_j)- k*rf ==1)
    
for d,i in index_di:
    MON.add_constraint(ksv_di[d,i]>= -MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j)+ MON.sum(big_mu_j[j]*v_j[j] for j in index_j))
                                    
for d in index_d:
    MON.add_constraint(kt_d[d]+ 1/((1-beta)*si)*MON.sum(kcv_di[d,i] for i in index_i) <= M*B1_d[d])
                                   
for d,i in index_di:
    MON.add_constraint(kcv_di[d,i]>= k*tv_di[d,i]- MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j) - kt_d[d])
                                      
MON.add_constraint(MON.sum(B1_d[d] for d in index_d) <= sd*alpha1)    

for d,i in index_di:
     MON.add_constraint(kem_di[d,i] >= MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j)-MON.sum(mu_dj[d,j]*v_j[j] for j in index_j))
        
for d,i in index_di:
     MON.add_constraint(kem_di[d,i] >= MON.sum(mu_dj[d,j]*v_j[j] for j in index_j)-MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j))

for d in index_d:
    MON.add_constraint(-M*B2_d[d] <= k*t_EMAD[d] - (emad/si)*MON.sum(kem_di[d,i] for i in index_i)+ MON.sum(mu_dj[d,j]*v_j[j] for j in index_j))
    
MON.add_constraint(MON.sum(B2_d[d] for d in index_d) <= sd*alpha2) 

for d in index_d:
    MON.add_constraint(MON.sum((1+ gamma_j[j])*v_j[j] for j in index_j) == W0*k)


    
MON.minimize((1/(si*sd))*MON.sum(ksv_di[d,i]*ksv_di[d,i] for d,i in index_di)+100*alpha1 + alpha2)


MON.parameters.mip.tolerances.mipgap.set(float(0.05))

MON.print_information()
msol = MON.solve(clean_before_solve=True)
# assert msol is not None, "model can't solve"
MON.report()     

In [ ]:
WEALTH = []
WEIGHT = []
w_amount= np.array([v_j[j].solution_value/k.solution_value for j in index_j])
final_account = assets.iloc[:,1:].copy()
final_account["risk_free"] = 0.03/365*7 
###########!!!!##############
WEIGHT.append(w_amount) # add beginning weight amounts 
WEALTH.append(W0) # add beginning wealthy
###########!!!!##############

######## hold this strategy at end of stage1 (before adjustment )
W0 = np.array((1+final_account.iloc[e,:]).tolist()) @ w_amount
x0_j = np.array((1+final_account.iloc[e,:]).tolist()) * w_amount

In [ ]:
for tms in range(1,81):
    Time_window = tms
    b = 0 + Time_window
    e = 127 + Time_window
    data = assets.iloc[b:e,:]
    numbers = 1000
    exo = TMCADS.iloc[b:e]
    exo_predict = TMCADS.iloc[e]
    N = 20 # prefer to make numbers/N = int
    Cs = 50

    # s1: Mean_reverting weighted 
    scenario1 = ptd.scenario_generation.Mean_reverting_secnario(data,numbers, weit = "index", rstate = rs)

    # s2: Mean_reverting non-weighted
    scenario2 = ptd.scenario_generation.Mean_reverting_secnario(data,numbers, weit = None, rstate = rs)

    # s3: GARCH(1,1) weighted
    scenario3 = ptd.scenario_generation.CC_MGARCH_11(data,numbers,weit = 'index', rstate = rs)

    # s4: GARCH(1,1) non-weighted
    scenario4 = ptd.scenario_generation.CC_MGARCH_11(data,numbers,weit = None, rstate = rs)

    # s5: CC_GJR_GARCH(1,1,1) weighted
    scenario5 = ptd.scenario_generation.CC_GJR_GARCH(data,numbers, weit = 'index', rstate = rs)

    # s6: CC_GJR_GARCH(1,1,1) non-weighted
    scenario6 = ptd.scenario_generation.CC_GJR_GARCH(data,numbers, weit = None, rstate = rs)

    # s7: CC_exo_GJR_GARCH(1,1,1) non-weighted
    scenario7 = ptd.scenario_generation.exo_GJR_GARCH(data,numbers,exo,exo_predict,weit = 'index', rstate = rs)

    # s8: CC_exo_GJR_GARCH(1,1,1) non-weighted
    scenario8 = ptd.scenario_generation.exo_GJR_GARCH(data,numbers,exo,exo_predict,weit = None , rstate = rs)

    #s9: multi-normal distributino
    scenario9 = ptd.scenario_generation.Multi_norm_asset(data, numbers,rstate = rs)

    #S10: multi-t distribution
    scenario10 = ptd.scenario_generation.Multi_t_asset(data, numbers, rstate = rs, dfr = 4)

    #S11: historial sampling
    scenario11 = ptd.scenario_generation.historial_sampling(data,numbers,weit = None, rstate = rs)

    #s12: last sampling
    scenario12 = ptd.scenario_generation.sample_lastN(data,N )

    scenario12 = pd.concat([scenario12]*math.ceil(numbers/N),ignore_index = True)


    emad =0.25
    q_var = 0.7

    Scenarios_set = []
    risk_measure = []
    mean_value = []
    for i in range(1,13):
        kmeans = KMeans(n_clusters= Cs, random_state=1).fit(eval('scenario'+str(i)))
        clusters = kmeans.cluster_centers_
        clusters = pd.DataFrame(clusters,columns = eval('scenario'+str(i)).columns)
        # add risk_free asset in portfolio 
        clusters["risk_free"] = 0.03/365*7
        Scenarios_set.append(clusters.values.tolist())

        # add the risk measure
        risk_measure.append(ptd.processing.risk_mean_function(clusters.iloc[:,0],emad,q_var).values.tolist())

        # add the mean value 
        mean_value.append(clusters.mean().values.tolist())

    return_target_set = np.array(Scenarios_set)[:,:,0]
    Scenarios_set = np.array(Scenarios_set)[:,:,1:] # shape d,i,j (data set,scenario,asset)
    risk_measure = np.array(risk_measure) # shape d,i,j (data set,risk_measure,1)
    target_mean_value = np.array(mean_value)[:,0] # shape d,j (data set,asset)
    assets_mean_value = np.array(mean_value)[:,1:] 

    target_ratio = []
    for i in range(risk_measure.shape[0]):
        if target_mean_value[i] <= 0:
            target_ratio.append(100000)
        else:
            target_ratio.append(((-risk_measure[:,0,0] +  risk_measure[:,2,0])/emad)[i]/target_mean_value[i])


    #####################################################################################################################

    X0_j = {(j):x0_j[j] for j in index_j}

    #####################################################################################################################

    # input the initial wealth 
    # (if we make decision variable become amount rahter than weight, then we need to multiple them by initial welath to get correct target)
    gamma = 0.001 # transaction cost

    # big M for chance constraint programming
    M = 10000
    beta = 0.7

    T_sv = risk_measure[:,1,0] # "semivariance_loss"
    T_EMAD = risk_measure[:,2,0] # "EMAD_loss"
    T_VAR = risk_measure[:,3,0] # "VAR_loss"
    T_CVAR = risk_measure[:,4,0] # "CVAR_loss"

    # the number of data set 
    sd = Scenarios_set.shape[0]
    # the number of scenario in data set
    si = Scenarios_set.shape[1]
    # the number of asset
    sj = Scenarios_set.shape[2]

    # transaction cost
    gamma_j = ([gamma]*(sj - 1))
    gamma_j.append(0)

    #(d,i,j)
    index_dij =list(product(range(sd), range(si),range(sj)))
    # (d,i)
    index_di =list(product(range(sd), range(si)))
    # （d,j)
    index_dj =list(product(range(sd), range(sj)))
    # （i,j)
    index_ij =list(product(range(si), range(sj)))
    #(d)
    index_d = [i for i in range(sd)]
    # (i)
    index_i = [i for i in range(si)]
    #(j)
    index_j = [i for i in range(sj)]

    # mu_dj 
    mu_dj = {(d,j): assets_mean_value[d,j] for d,j in index_dj}

    # return_rate(d,i,j)
    r_dij = {(d,i,j): Scenarios_set[d,i,j] for d,i,j in index_dij}

    # target value of portfolio of return in scenario d of asset i 
    tv_di = {(d,i): W0*return_target_set[d,i] for d,i in index_di}

    # the shape of risk measure is (d,n_th_risk_measure,1)
    #the sequence of output of risk measure ["mean_loss","semivariance_loss","EMAD_loss","VAR_loss","CVAR_loss"]

    t_sv = {(d): T_sv[d]*(W0**2) for d in index_d} # semivariance_loss"
    t_EMAD = {(d): T_EMAD[d]*W0 for d in index_d}   # "EMAD_loss"
    t_VAR = {(d): T_VAR[d]*W0 for d in index_d} # "VAR_loss"
    t_CVAR = {(d): T_CVAR[d]*W0 for d in index_d} # "CVAR_loss"

    ##############################################################################################################################

    # Declare and initialize model
    MON = Model(name = 'MO_N',log_output=True)

    ### decision variables

    #the amount invested in asset j
    k =  MON.continuous_var(lb=0.0, ub=float('inf'))
    v_j =  MON.continuous_var_dict(index_j, lb=0.0, ub=float('inf'))
    # auxiliary variable of MAD in data set d, scenario i
    kem_di = MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
    # auxiliary variable of CVaR in data set d, scenario i 
    kcv_di =  MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
    # auxiliary variable of semivariance in data set d, scenario i
    ksv_di = MON.continuous_var_dict(index_di, lb=0.0, ub=float('inf'))
    # the auxiliary variable of VaR 
    kt_d = MON.continuous_var_dict(index_d, lb = -float('inf'),ub=float('inf'))
    B1_d = MON.integer_var_dict(index_d,0,1)
    B2_d = MON.integer_var_dict(index_d,0,1)

    v_jb = MON.continuous_var_dict(index_j, lb=0.0, ub=float('inf'))
    v_js = MON.continuous_var_dict(index_j, lb=0.0, ub=float('inf'))

    alpha1 = MON.continuous_var(lb=0.0,ub=float('inf'))
    alpha2 = MON.continuous_var(lb=0.0,ub=float('inf'))
    rf = -W0*2*gamma


    for d in index_d:
        MON.add_constraint(MON.sum(big_mu_j[j]*v_j[j] for j in index_j)- k*rf ==1)

    for d,i in index_di:
        MON.add_constraint(ksv_di[d,i]>= -MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j)+ MON.sum(big_mu_j[j]*v_j[j] for j in index_j))

    for d in index_d:
        MON.add_constraint(kt_d[d]+ 1/((1-beta)*si)*MON.sum(kcv_di[d,i] for i in index_i) <= M*B1_d[d])

    for d,i in index_di:
        MON.add_constraint(kcv_di[d,i]>= k*tv_di[d,i]- MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j) - kt_d[d])

    MON.add_constraint(MON.sum(B1_d[d] for d in index_d) <= sd*alpha1)    

    for d,i in index_di:
         MON.add_constraint(kem_di[d,i] >= MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j)-MON.sum(mu_dj[d,j]*v_j[j] for j in index_j))

    for d,i in index_di:
         MON.add_constraint(kem_di[d,i] >= MON.sum(mu_dj[d,j]*v_j[j] for j in index_j)-MON.sum(r_dij[d,i,j]*v_j[j] for j in index_j))

    for d in index_d:
        MON.add_constraint(-M*B2_d[d] <= k*t_EMAD[d] - (emad/si)*MON.sum(kem_di[d,i] for i in index_i)+ MON.sum(mu_dj[d,j]*v_j[j] for j in index_j))

    MON.add_constraint(MON.sum(B2_d[d] for d in index_d) <= sd*alpha2) 

    for j in index_j:
        MON.add_constraint(k*X0_j[j] + v_jb[j] - v_js[j] == v_j[j])
        MON.add_constraint(k*X0_j[j]>= v_js[j])

    MON.add_constraint(MON.sum((1 + gamma_j[j])*v_jb[j] for j in index_j) == MON.sum((1 - gamma_j[j])*v_js[j] for j in index_j) )

    MON.minimize((1/(si*sd))*MON.sum(ksv_di[d,i]*ksv_di[d,i] for d,i in index_di)+100*alpha1 + alpha2)

    #############################################################################################################################

    MON.parameters.mip.tolerances.mipgap.set(float(0.05))

    MON.print_information()
    msol = MON.solve(clean_before_solve=True)
    # assert msol is not None, "model can't solve"
    MON.report()     

    ##############################################################################################################################

    w_amount= np.array([v_j[j].solution_value/k.solution_value for j in index_j])
    final_account = assets.iloc[:,1:].copy()
    final_account["risk_free"] = 0.03/365*7 
    ###########!!!!##############
    WEIGHT.append(w_amount) # add beginning weight amounts 
    WEALTH.append(w_amount.sum()) # add beginning wealthy
    ###########!!!!##############

    ######## hold this strategy at end of stage1 (before adjustment )
    W0 = np.array((1+final_account.iloc[e,:]).tolist()) @ w_amount
    x0_j = np.array((1+final_account.iloc[e,:]).tolist()) * w_amount

    ##############################################################################################################

    
    pd.DataFrame(WEALTH).to_csv("NASDAQ_wealth_ratio.csv")
    pd.DataFrame(WEIGHT).to_csv("NASDAQ_weight_ratio.csv")

###########!!!!##############
WEIGHT.append(x0_j) # add beginning weight amounts 
WEALTH.append(W0) # add beginning wealthy

pd.DataFrame(WEALTH).to_csv("NASDAQ_wealth_ratio.csv")
pd.DataFrame(WEIGHT).to_csv("NASDAQ_weight_ratio.csv")